<a href="https://colab.research.google.com/github/ganeshan007/Video-Classification-using-keras/blob/master/video_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import os
cam = cv2.VideoCapture("/content/v_swing_01_01.avi") 
  
try: 
      
    # creating a folder named data 
    if not os.path.exists('data'): 
        os.makedirs('data') 
  
# if not created then raise error 
except OSError: 
    print ('Error: Creating directory of data') 
  
# frame 
currentframe = 0
  
while(True):
  
      
    # reading from frame 
    ret,frame = cam.read() 
  
    if ret: 
        # if video is still left continue creating images 
        name = './data/frame' + str(currentframe) + '.jpg'
        print ('Creating...' + name) 
  
        # writing the extracted images 
        cv2.imwrite(name, frame) 
  
        # increasing counter so that it will 
        # show how many frames are created 
        currentframe += 1
    else: 
        break
  
# Release all space and windows once done 
cam.release() 
cv2.destroyAllWindows() 

In [0]:
! wget https://drive.google.com/open?id=14aL5ri0t7UIq_7KzdgazILyyu0SxBhEy

In [0]:
 !unzip open?id=14aL5ri0t7UIq_7KzdgazILyyu0SxBhEy.1

In [0]:
! ls Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
! pip install patool
import patoolib
patoolib.extract_archive("/content/drive/My Drive/action_youtube_naudio.rar", outdir="/content/drive/My Drive/")

In [0]:
path = '/content/drive/My Drive/action_youtube_naudio/tennis_swing'
import cv2
import math
import os
currentframe = 0
count=0
#/content/drive/My Drive/action_youtube_naudio/basketball/v_spikingspiking_01/v_spiking_01_01.avi
l1=['v_tennis_01','v_tennis_02','v_tennis_03','v_tennis_04','v_tennis_05','v_tennis_06','v_tennis_07','v_tennis_08','v_tennis_09','v_tennis_10','v_tennis_11','v_tennis_12','v_tennis_13','v_tennis_14','v_tennis_15','v_tennis_16','v_tennis_17','v_tennis_18','v_tennis_19','v_tennis_20','v_tennis_21','v_tennis_22','v_tennis_23','v_tennis_24','v_tennis_25']
for i in l1:
    p=os.path.join(path,i)
    for dirName, subdirList, fileList in os.walk(p):
      for file in fileList:
#         print(p+'/'+file)
        cam=cv2.VideoCapture(p+'/'+file)
        fps = int(cam.get(cv2.CAP_PROP_FPS))
#         print(fps)
        try: 
      
    # creating a folder named data 
          if not os.path.exists('tennis_swing'): 
            os.makedirs('tennis_swing') 
  
# if not created then raise error 
        except OSError: 
          print ('Error: Creating directory of data')
      
        
  
        while(True):
      
            # reading from frame 
            ret,frame = cam.read() 

            if ret:
                #cam.set(cv2.CAP_PROP_POS_MSEC,(count*1000))
                # if video is still left continue creating images 
                name = './tennis_swing/frame' + str(currentframe) + '.jpg'
                print ('Creating...' + name)
                if (currentframe%(math.floor(fps))) == 0 :
                  cv2.imwrite(name,frame)

                # writing the extracted images 
#                 cv2.imwrite(name, frame)
#                 count+=1

                # increasing counter so that it will 
                # show how many frames are created 
                currentframe += 1
            else: 
                break

        # Release all space and windows once done 
        cam.release() 
        cv2.destroyAllWindows()

In [0]:
#@title
from google.colab import files
!zip -r /content/tennis_swing.zip  /content/tennis_swing
files.download("/content/tennis_swing.zip")

In [0]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os



# initialize the set of labels from the spots activity dataset we are
# going to train our network on
LABELS = set(["basketball", "biking", "diving","golf_swing","horse_riding","soccer_juggling","swing","tennis_swing","trampoline_jumping","volleyball_spiking","walking"])

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
folderPaths = '/content/drive/My Drive/youtube_data'
data = []
labels = []
for i in LABELS:                                       
  p1=os.path.join(folderPaths,i)
  for dirName, subdirList, fileList in os.walk(p1):
      for file in fileList:
#         print(p1+'/'+file)
        imagePath=p1+'/'+file
#         print(imagePath)
        label=imagePath.split(os.path.sep)[-2]
        image=cv2.imread(imagePath)
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image=cv2.resize(image,(224,224))
        

#         print(label)
        if label not in LABELS:
          continue
        data.append(image)
        labels.append(label)
#         np.append(data,image)
#         np.append(labels,label)
#         print(labels)
#         print(data)
#         print(labels)
data=np.array(data)
labels=np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, stratify=labels, random_state=42)
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu",kernel_initializer="he_normal")(headModel)
headModel = Dense(256,activation="relu",kernel_initializer="he_normal")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)


model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
	layer.trainable = False

# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = Adam(lr=1e-4)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=32),
	steps_per_epoch=len(trainX) // 32,
	validation_data=valAug.flow(testX, testY),
	validation_steps=len(testX) // 32,
	epochs=50)

print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

# plot the training loss and accuracy
N = 50
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")


model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

# serialize the label binarizer to disk
f = open(output/lb.pickle, "wb")
f.write(pickle.dumps(lb))
f.close()

